# Evolutionary Computation for Evolving Neural Network EMG Classifier Structure and Weights
In this project, the goal is to improve the structure and weights of a neural network that classifies EMG signals, and compare the genetic solutions to standard classifiers.

Controls:
- Random Forest Classifier
- 